In [10]:
!python -V

Python 3.9.18


In [11]:
import pandas as pd

In [12]:
import pickle

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [15]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/yunbo/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1710747964467, experiment_id='1', last_update_time=1710747964467, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [18]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [19]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [20]:
len(df_train), len(df_val)

(73908, 61921)

In [21]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [22]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715204520257

In [26]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [28]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [29]:
import xgboost as xgb

In [30]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [31]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [35]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [36]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:10:42] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.63051                          
[1]	validation-rmse:9.45642                           
[2]	validation-rmse:8.59687                           
[3]	validation-rmse:7.98297                           
[4]	validation-rmse:7.54633                           
[5]	validation-rmse:7.24210                           
[6]	validation-rmse:7.02564                           
[7]	validation-rmse:6.87547                           
[8]	validation-rmse:6.76593                           
[9]	validation-rmse:6.68959                           
[10]	validation-rmse:6.63341                          
[11]	validation-rmse:6.59121                          
[12]	validation-rmse:6.55977                          
[13]	validation-rmse:6.53660                          
[14]	validation-rmse:6.51821                          
[15]	validation-rmse:6.50425                          
[16]	validation-rmse:6.49322                          
[17]	validation-rmse:6.48295                          
[18]	valid

/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:11:11] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:10.60851                                                   
[4]	validation-rmse:10.28052                                                   
[5]	validation-rmse:9.97817                                                    
[6]	validation-rmse:9.69973                                                    
[7]	validation-rmse:9.44371                                                    
[8]	validation-rmse:9.20863                                                    
[9]	validation-rmse:8.99302                                                    
[10]	validation-rmse:8.79549                                                   
[11]	validation-rmse:8.61455                                                   
[12]	validation-rmse:8.44907                                                   
[13]	validation-rmse:8.29791                                                   
[14]	validation-rmse:8.16007                                                   
[15]	validation-rmse:8.03444            

...

[6]	validation-rmse:9.73418                                                     
[7]	validation-rmse:9.48019                                                     
[8]	validation-rmse:9.24687                                                     
[9]	validation-rmse:9.03301                                                     
[10]	validation-rmse:8.83618                                                    
[11]	validation-rmse:8.65583                                                    
[12]	validation-rmse:8.49108                                                    
[13]	validation-rmse:8.34009                                                    
[14]	validation-rmse:8.20200                                                    
[15]	validation-rmse:8.07619                                                    
[16]	validation-rmse:7.96154                                                    
[17]	validation-rmse:7.85692                                                    
[18]	validation-rmse:7.76209

/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:36:41] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:6.85794                                                     
[4]	validation-rmse:6.78228                                                     
[5]	validation-rmse:6.74262                                                     
[6]	validation-rmse:6.72489                                                     
[7]	validation-rmse:6.71350                                                     
[8]	validation-rmse:6.70751                                                     
[9]	validation-rmse:6.70151                                                     
[10]	validation-rmse:6.69909                                                    
[11]	validation-rmse:6.69666                                                    
[12]	validation-rmse:6.69341                                                    
[13]	validation-rmse:6.68919                                                    
[14]	validation-rmse:6.68598                                                    
[15]	validation-rmse:6.68087

/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:36:59] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.77945                                                    
[1]	validation-rmse:9.68425                                                     
[2]	validation-rmse:8.85775                                                     
[3]	validation-rmse:8.24207                                                     
[4]	validation-rmse:7.78946                                                     
[5]	validation-rmse:7.45855                                                     
[6]	validation-rmse:7.21766                                                     
[7]	validation-rmse:7.04008                                                     
[8]	validation-rmse:6.91188                                                     
[9]	validation-rmse:6.81653                                                     
[10]	validation-rmse:6.74697                                                    
[11]	validation-rmse:6.69572                                                    
[12]	validation-rmse:6.65369

In [39]:
best_params = {
    'learning_rate': 0.11558775676492819,
    'max_depth': 49,
    'min_child_weight': 1.3913919084738615,
    'objective': 'reg:linear',
    'reg_alpha': 0.35940520308672946,
    'reg_lambda': 0.09603619965585856,
    'seed': 42
}

mlflow.xgboost.autolog()



booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)


2024/03/18 10:49:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '41f3039d3ed5403f9d30a0186672b6fb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:49:48] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.28055
[1]	validation-rmse:10.48706
[2]	validation-rmse:9.81640
[3]	validation-rmse:9.25292
[4]	validation-rmse:8.77872
[5]	validation-rmse:8.38541
[6]	validation-rmse:8.05802
[7]	validation-rmse:7.78932
[8]	validation-rmse:7.56436
[9]	validation-rmse:7.38109
[10]	validation-rmse:7.23066
[11]	validation-rmse:7.10681
[12]	validation-rmse:7.00336
[13]	validation-rmse:6.91772
[14]	validation-rmse:6.84847
[15]	validation-rmse:6.78996
[16]	validation-rmse:6.74116
[17]	validation-rmse:6.70106
[18]	validation-rmse:6.66667
[19]	validation-rmse:6.63853
[20]	validation-rmse:6.61374
[21]	validation-rmse:6.59305
[22]	validation-rmse:6.57537
[23]	validation-rmse:6.55982
[24]	validation-rmse:6.54666
[25]	validation-rmse:6.53387
[26]	validation-rmse:6.52463
[27]	validation-rmse:6.51517
[28]	validation-rmse:6.50701
[29]	validation-rmse:6.50019
[30]	validation-rmse:6.49398
[31]	validation-rmse:6.48864
[32]	validation-rmse:6.48330
[33]	validation-rmse:6.47881
[34]	validation-rmse:6

2024/03/18 10:50:48 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/03/18 10:50:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:50:48] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


In [38]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.11558775676492819,
        'max_depth': 49,
        'min_child_weight': 1.3913919084738615,
        'objective': 'reg:linear',
        'reg_alpha': 0.35940520308672946,
        'reg_lambda': 0.09603619965585856,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:39:49] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.28055
[1]	validation-rmse:10.48706
[2]	validation-rmse:9.81640
[3]	validation-rmse:9.25292
[4]	validation-rmse:8.77872
[5]	validation-rmse:8.38541
[6]	validation-rmse:8.05802
[7]	validation-rmse:7.78932
[8]	validation-rmse:7.56436
[9]	validation-rmse:7.38109
[10]	validation-rmse:7.23066
[11]	validation-rmse:7.10681
[12]	validation-rmse:7.00336
[13]	validation-rmse:6.91772
[14]	validation-rmse:6.84847
[15]	validation-rmse:6.78996
[16]	validation-rmse:6.74116
[17]	validation-rmse:6.70106
[18]	validation-rmse:6.66667
[19]	validation-rmse:6.63853
[20]	validation-rmse:6.61374
[21]	validation-rmse:6.59305
[22]	validation-rmse:6.57537
[23]	validation-rmse:6.55982
[24]	validation-rmse:6.54666
[25]	validation-rmse:6.53387
[26]	validation-rmse:6.52463
[27]	validation-rmse:6.51517
[28]	validation-rmse:6.50701
[29]	validation-rmse:6.50019
[30]	validation-rmse:6.49398
[31]	validation-rmse:6.48864
[32]	validation-rmse:6.48330
[33]	validation-rmse:6.47881
[34]	validation-rmse:6

/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [10:40:30] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [40]:
import mlflow
logged_model = 'runs:/41f3039d3ed5403f9d30a0186672b6fb/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.xgboost.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
y = loaded_model.predict(valid)
y[:20]

/Users/yunbo/anaconda3/envs/basic/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [11:10:15] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


array([14.293808 ,  6.8635406, 14.469265 , 24.46155  ,  9.345045 ,
       17.146893 , 10.557908 ,  8.365253 ,  9.519408 , 18.608387 ,
       11.337758 , 16.273603 ,  8.86146  ,  5.844951 , 17.037682 ,
       10.301903 , 10.891236 ,  9.453916 ,  5.4378595,  8.083388 ],
      dtype=float32)